## Embedding model 의 문장 유사도 테스트
### [KoSimCSE-supervised-roberta-large](https://huggingface.co/daekeun-ml/KoSimCSE-supervised-roberta-large)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor

from transformers import AutoConfig, PretrainedConfig, PreTrainedModel
from transformers import AutoModel, AutoTokenizer, logging

class SimCSEConfig(PretrainedConfig):
    def __init__(self, version=1.0, **kwargs):
        self.version = version
        super().__init__(**kwargs)

class SimCSEModel(PreTrainedModel):
    config_class = SimCSEConfig

    def __init__(self, config):
        super().__init__(config)
        self.backbone = AutoModel.from_pretrained(config.base_model)
        self.hidden_size: int = self.backbone.config.hidden_size
        self.dense = nn.Linear(self.hidden_size, self.hidden_size)
        self.activation = nn.Tanh()

    def forward(
        self,
        input_ids: Tensor,
        attention_mask: Tensor = None,
        # RoBERTa variants don't have token_type_ids, so this argument is optional
        token_type_ids: Tensor = None,
    ) -> Tensor:
        # shape of input_ids: (batch_size, seq_len)
        # shape of attention_mask: (batch_size, seq_len)
        outputs: BaseModelOutputWithPoolingAndCrossAttentions = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        emb = outputs.last_hidden_state[:, 0]

        if self.training:
            emb = self.dense(emb)
            emb = self.activation(emb)

        return emb

# Load pre-trained model
model = SimCSEModel.from_pretrained("daekeun-ml/KoSimCSE-supervised-roberta-large")
tokenizer = AutoTokenizer.from_pretrained("daekeun-ml/KoSimCSE-supervised-roberta-large")


/opt/homebrew/Caskroom/miniforge/base/envs/embedding_test/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 582/582 [00:00<00:00, 1.06MB/s]
model.safetensors: 100%|██████████| 1.35G/1.35G [01:04<00:00, 20.9MB/s]
config.json: 100%|██████████| 547/547 [00:00<00:00, 1.35MB/s]
model.safetensors: 100%|██████████| 1.35G/1.35G [00:24<00:00, 54.2MB/s]
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
tokenizer_config.json: 100%|██████████| 415/415 [00:00<00:00, 561kB/s]
vocab.txt: 100%|██████████| 248k/248k [00:00<00:00, 507kB/s]
tokenizer.j

tensor([[92.9861]], grad_fn=<MulBackward0>) tensor([[83.0575]], grad_fn=<MulBackward0>)


In [10]:
def show_embedding_score(tokenizer, model, sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    embeddings = model(**inputs)
    score01 = cal_score(embeddings[0,:], embeddings[1,:])
    score02 = cal_score(embeddings[0,:], embeddings[2,:])
    score03 = cal_score(embeddings[1,:], embeddings[2,:])
    
    score = [score01, score02, score03 ]
    max_score = max(score).item()
    max_score_idx = score.index(max(score))
    
    if max_score_idx == 0:
        print(f"1,2번 문장이 {max_score}로 가장 유사합니다.")
    elif max_score_idx == 1:  
        print(f"1,3번 문장이 {max_score}로 가장 유사합니다.")
    else:    
        print(f"2,3번 문장이 {max_score}로 가장 유사합니다.") 

    print('(', score01.item(), score02.item(), score03.item(), ')')

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)
    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100 

In [12]:
# Inference example
sentences = ['이번 주 일요일에 분당 이마트 점은 문을 여나요?',
             '일요일에 분당 이마트는 문 열어요?',
             '분당 이마트 점은 토요일에 몇 시까지 하나요']

show_embedding_score(tokenizer, model, sentences)


1,2번 문장이 92.98614501953125로 가장 유사합니다.
( 92.98614501953125 83.0574722290039 77.81117248535156 )


## txt 파일로 저장되어있는 query set에 대한 유사도 테스트

In [6]:
from ast import literal_eval

with open('query.txt', 'r') as file:
    
    file_contents = file.read()
    query_list = literal_eval(file_contents)

query_list


[['치타가 들판을 가로 질러 먹이를 쫓는다.', '치타 한 마리가 먹이 뒤에서 달리고 있다.', '원숭이 한 마리가 드럼을 연주한다.'],
 ['이번 주 일요일에 분당 이마트 점은 문을 여나요?',
  '일요일에 분당 이마트는 문 열어요?',
  '분당 이마트 점은 토요일에 몇 시까지 하나요?'],
 ['빠른 갈색 여우가 게으른 개를 뛰어넘는다', '날쌘 갈색 여우가 졸린 개를 넘어간다', '태양은 동쪽에서 뜨고 서쪽에서 진다'],
 ['인공지능은 세계를 변화시키고 있다', 'AI는 우리 행성을 혁신하고 있다', '사과는 맛있고 영양가 있는 과일이다'],
 ['파이썬은 인기 있는 프로그래밍 언어이다',
  '많은 개발자들이 소프트웨어 개발을 위해 파이썬을 사용한다',
  '코끼리는 가장 큰 육상 동물이다'],
 ['커피는 인기 있는 음료이다', '많은 사람들이 아침에 커피를 마신다', '산에는 여러 종류의 나무가 있다'],
 ['데이터 과학은 정보를 분석하는 분야이다',
  '많은 회사들이 데이터 분석을 위해 데이터 과학자를 고용한다',
  '바다는 광대하고 신비로운 곳이다'],
 ['서울은 대한민국의 수도이다', '서울에는 다양한 문화와 역사가 존재한다', '사막은 건조하고 모래가 많은 지역이다'],
 ['성남에는 편의점이 몇 개가 있을까?', '너가 자주 가는 편의점은 성남에 있니?', '서울은 편의시설이 얼마나 많은가?'],
 ['컴퓨터 프로그래밍은 창의적인 과정이다',
  '많은 사람들이 소프트웨어 개발을 배운다',
  '산책하며 신선한 공기를 마시는 것은 건강에 좋다'],
 ['수학은 문제 해결에 중요하다', '공학 분야에서 수학은 필수적이다', '영화 감상은 좋은 여가 활동이다'],
 ['건강한 식단은 삶의 질을 향상시킨다', '많은 사람들이 영양가 있는 음식을 찾는다', '별을 관찰하는 것은 매력적인 취미이다'],
 ['운동은 몸과 마음에 좋다', '규칙적인 운동은 건강을 유지하는데 도움이 된다', '좋은 책을 읽는 것은 지식을 넓히는데 도움이 

In [19]:
for i, query in enumerate(query_list):
    print(f'{i})\n{query}')
    show_embedding_score(tokenizer, model, query)
    print() 
    

0)
['치타가 들판을 가로 질러 먹이를 쫓는다.', '치타 한 마리가 먹이 뒤에서 달리고 있다.', '원숭이 한 마리가 드럼을 연주한다.']
1,2번 문장이 83.16889953613281로 가장 유사합니다.
( 83.16889953613281 24.935367584228516 27.913841247558594 )

1)
['이번 주 일요일에 분당 이마트 점은 문을 여나요?', '일요일에 분당 이마트는 문 열어요?', '분당 이마트 점은 토요일에 몇 시까지 하나요?']
1,2번 문장이 92.98614501953125로 가장 유사합니다.
( 92.98614501953125 83.68720245361328 78.30581665039062 )

2)
['빠른 갈색 여우가 게으른 개를 뛰어넘는다', '날쌘 갈색 여우가 졸린 개를 넘어간다', '태양은 동쪽에서 뜨고 서쪽에서 진다']
1,2번 문장이 76.48316955566406로 가장 유사합니다.
( 76.48316955566406 32.25828170776367 31.092655181884766 )

3)
['인공지능은 세계를 변화시키고 있다', 'AI는 우리 행성을 혁신하고 있다', '사과는 맛있고 영양가 있는 과일이다']
1,2번 문장이 87.18400573730469로 가장 유사합니다.
( 87.18400573730469 36.52079772949219 38.64492416381836 )

4)
['파이썬은 인기 있는 프로그래밍 언어이다', '많은 개발자들이 소프트웨어 개발을 위해 파이썬을 사용한다', '코끼리는 가장 큰 육상 동물이다']
1,2번 문장이 90.1249771118164로 가장 유사합니다.
( 90.1249771118164 32.72272491455078 28.334888458251953 )

5)
['커피는 인기 있는 음료이다', '많은 사람들이 아침에 커피를 마신다', '산에는 여러 종류의 나무가 있다']
1,2번 문장이 69.57415008544922로 가장 유사합니다.
( 69.5741